In [1]:
import qrml
import numpy                as np
import matplotlib.pyplot    as plt

%matplotlib qt

**Note:** This notebook gives further clarification on the functions and parameters of `qrml`; however for full details, please refer to the paper [[1]](#1).

### Pointcloud

We take a closer look at `qrml` applied to the real projective plane. We take the standard embedding into $\mathbb{R}^4$ given by:
$$ S^2\to \mathbb{R}^4 : (x, y, z) \mapsto (xy, xz, y^2 − z^2, 2yz)$$
We generate our pointcloud by uniformly sampling 1000 points from $S^2$ (found in `../datasets/sphere.txt`) and applying the above embedding .

In [2]:
n_points = 1000
sphere = np.loadtxt('../datasets/sphere.txt')
x0 = sphere[:, 0] * sphere[:, 1]
x1 = sphere[:, 0] * sphere[:, 2]
x2 = sphere[:, 1]**2 - sphere[:, 2]**2
x3 = 2 * sphere[:, 1] * sphere[:, 2]
rp2 = np.stack([x0, x1, x2, x3], axis=-1)

### Projection

We follow the same steps as in the README.md file to compute a 1-skeleton on our data and its projection.

In [3]:
params_rp2 = {'S1':0.2, 'k':10, 'threshold_var':0.08, 'edge_sen':1, 'k0':100}

S_rp2 = qrml.Simplex()
S_rp2.build_simplex(rp2, **params_rp2)
S_rp2.normal_coords(two_d=False, **params_rp2)

Restricted license - for non-production use only - expires 2023-10-25


Full explanations of the parameters `k`, `threshold_var` and `edge_sen` can be found in the paper [[2]](#2) under the "naive" algorithm; these parameters control the construction of the 1-skeleton on our data under `build_simplex`. From this 1-skeleton, we compute the estimated dimension of our pointcloud.

Roughly, `k` is the parameter for the KNN 1-skeleton that we start with and that we refine through "visible" then "safe" edges. The parameters `threshold_var` and `edge_sen` control the quality of the edges that we take. Higher values of `threshold_var` and `edge_sen` result in more edges being kept. The standard values the paper [[2]](#2) recommends are `k=10, threshold_var=0.08, edge_sen=1`.

To estimate the intrinsic dimension of our pointcloud, for each point, we take the set of its neighbouring points (in the computed 1-skeleton) and apply PCA. We take the local dimension of this neighbourhood to be the "knee" point of the curve given by `PCA.explained_variance_ratio_` from `scikit-learn`. We use the function `KneeLocator` from the python package `kneed`, which takes the parameter `S1`, to locate the "knee" point; we take `S1=0.2` as default. The overall estimated dimension is then the mode of all the local estimated dimensions.

Our implementation introduces the extra parameter `k0` for the computation of the projection under `normal_coords`. We found this necessary in order to make the algorithm to work. Originally, in order to compute the projection of a generic point, we find the shortest path from the base point (of our projection) to our generic point by Dijkstra's algorithm. We take the neighbouring points of the predecessor to the generic point on this path, whose projections are already computed, and use this local information to compute the projection of the generic point. We need at least the estimated dimension number of neighbouring points to compute the projection.

We found in implementing this algorithm that we would regularly run into points, throughout the pointcloud, which do not have enough neighbouring points to correctly compute the projection. To correct this, we instead take the set of neighbouring points (which already have their projection computed) of the neighbouring points (which already have their projection computed) of the predecessor point (we exclude the predecessor point) and take the estimated dimension plus `k0` points which are closest to our generic point. Hence, `k0` controls how local we require our local data to be, in computing projections.

### Visualization

The method `show_boundary` allows us to visualize our projection, the boundary of our projection, how much projected points are displaced and how boundary points are connected to other boundary points.

In order to compute the boundary of our projection, we compute the alpha shape [[3]](#3) of our projected points with the parameter `alpha`. The implementation of alpha shape, that we use, was taken from [[4]](#4).

In [4]:
S_rp2.show_boundary(alpha=1, tol=2, c=rp2[:, 0], show_tear_points=True, a=2.5, show_connections=True, show_pointcloud=False, connection_tol=5, **params_rp2)

### Quotient

The method `plot_quotient` computes and plots the quotient identifications of our projection.

In [5]:
_ = S_rp2.plot_quotient(c=rp2[:, 0], alpha=1, tol=2, quotient_tol=15, tol1=5, show_pointcloud=False)

Break down the algo

Side example with sphere

Other examples

### References

<a id="1">[1]</a> 
TODO

<a id="2">[2]</a> 
Tong, L., Zha, H. Riemannian manifold learning. *IEEE Transactions on Pattern Analysis
and Machine Intelligence* 30.5 (2008): 796-809.

<a id="3">[3]</a>
*Alpha shape*, Wikipedia, viewed 30 August 2022,
<https://en.wikipedia.org/wiki/Alpha_shape>

<a id="4">[4]</a> 
Hanniel, I. 2018, *Calculate bounding polygon of alpha shape from the Delaunay triangulation*, Stack Overflow, viewed 30 August 2022,  
<https://stackoverflow.com/questions/23073170/calculate-bounding-polygon-of-alpha-shape-from-the-delaunay-triangulation>